## Importar librerias a utilizar

In [13]:
import pandas as pd

## Lectura de los datos

In [3]:
# Lectura de sitios dentro de Louisiana e Indiana (Google)
sitios= pd.read_parquet("../../Datasets/final-ds/sitios_combined_full") #Obtenido del archivo ETL/business_Google.ipynb

# Reviews Indiana
indiana= pd.read_parquet("../../Datasets/merge/ReviewIndiana")  # Obtenido del archivo ETL/estados

# Reviews Louisiana
louisiana= pd.read_parquet("../../Datasets/merge/ReviewLouisiana") # Obtenido del archivo ETL/estados

- Realizamos la union de los sitios con los estados que correspondan

In [4]:
indiana_sitios = indiana.merge(sitios,how="inner",on="gmap_id")
louisiana_sitios = louisiana.merge(sitios,how="inner",on="gmap_id")

- Observamos la cantidad de registros que quedaron en cada estado

In [16]:
louisiana_sitios.shape, indiana_sitios.shape

((609940, 14), (1126351, 14))

- Chequeo de valores nulos

In [19]:
louisiana_sitios.isna().sum()

user_id                  0
rating                   0
text                280034
gmap_id                  0
date                     0
name                     0
address               1260
latitude                 0
longitude                0
category               202
avg_rating               0
num_of_reviews           0
relative_results     13953
url                      0
dtype: int64

In [49]:
indiana_sitios = pd.read_parquet("../../Datasets/merge/indiana_sitios.parquet")
louisiana_sitios = pd.read_parquet("../../Datasets/merge/louisiana_sitios.parquet")

In [7]:
indiana_sitios.isna().sum()

user_id                  0
rating                   0
text                494064
gmap_id                  0
date                     0
name                     0
address               2735
latitude                 0
longitude                0
category               468
avg_rating               0
num_of_reviews           0
relative_results     30161
url                      0
dtype: int64

- Pasamos de `datetime` a `date`

In [5]:
# Louisiana
louisiana_sitios["date"] = louisiana_sitios["date"].dt.strftime("%Y-%m-%d").astype("date64[pyarrow]")

#Indiana
indiana_sitios["date"] = indiana_sitios["date"].dt.strftime("%Y-%m-%d").astype("date64[pyarrow]")

- Rellenamos los valores nulos de la columna `category` y la transformamos en string. Esto nos servira para el Analisis Exploratorio de Datos

In [14]:
louisiana_sitios.fillna({"category":"[]"},inplace=True)
indiana_sitios.fillna({"category":"[]"},inplace=True)

- Extraemos los primeros datos de la columna `category`

In [10]:
indiana_sitios["first_category"] = indiana_sitios["category"].apply(lambda x: x[0] if len(x)>=1 else "[]")
louisiana_sitios["first_category"] = louisiana_sitios["category"].apply(lambda x: x[0] if len(x)>=1 else "[]")

In [15]:
# Transformamos a string
louisiana_sitios["category"] = louisiana_sitios["category"].apply(lambda x: " ".join(x) if len(x)>0 else "")

In [16]:
indiana_sitios["category"] = indiana_sitios["category"].apply(lambda x: " ".join(x) if len(x)>0 else "")

- Filtramos para obtener una categoria principal

In [61]:
def checkCategory(word):
    if "Hotel" in word or "hotel" in word:
        return "Hotel"
    elif "Restaurant" in word or "restaurant" in word:
        return "Restaurant"
    else: return "Otros"
    
louisiana_sitios["main_category"] = louisiana_sitios["category"].apply(checkCategory)
indiana_sitios["main_category"] = louisiana_sitios["category"].apply(checkCategory)

- Realizamos un <i>merge</i> de las tablas para luego utilizar en la construccion de un `dashboard`

In [18]:
# Asignamos una columna extra "state" para diferenciar de que estado estamos hablando
louisiana_sitios["state"] = "Louisiana"
indiana_sitios["state"] = "Indiana"

In [19]:
google_states_reviews = pd.concat([indiana_sitios,louisiana_sitios])

In [20]:
google_states_reviews.sample(2)

,user_id,rating,text,gmap_id,date,name,address,latitude,longitude,category,avg_rating,num_of_reviews,relative_results,url,first_category,state
1060752,1.047858e+20,1,"Terrible food, terrible drinks",0x886b53af68572a35:0xf11f3494b0d95582,2018-02-15,Old Pro's Table,"Old Pro's Table, 827 Broad Ripple Ave, Indiana...",39.869564,-86.143949,Sports bar Bar Club Restaurant,4.2,168,"[0x886b53a58c67f7ff:0xf615ee6a359aa05d, 0x886b...",https://www.google.com/maps/place//data=!4m2!3...,Sports bar,Indiana
894569,1.049584e+20,5,What an amazing place! I was in a wedding host...,0x886bb3a362b62d3f:0xb38e96c4c86d3bed,2020-01-16,Willow Lake Event Center,"Willow Lake Event Center, 2130 W County Rd 200...",39.016134,-85.651866,Wedding venue Event venue,4.8,73,"[0x886b97149489ef2f:0x2fb8eb7c527e4edf, 0x886b...",https://www.google.com/maps/place//data=!4m2!3...,Wedding venue,Indiana


In [64]:
# Exportamos
google_states_reviews.to_parquet("../../Datasets/merge/google_states_reviews")

In [26]:
google_states_reviews["first_category"].unique()

array(['Security guard service', 'Entertainer', 'Pharmacy', ...,
       'Diamond dealer', 'European restaurant', 'Tattoo artist'],
      dtype=object)

- Exportamos a parquet los archivos de los estados separados para luego analizarlos en eL <I>EDA</I>

In [33]:
indiana_sitios.to_parquet("../../Datasets/merge/indiana_sitios.parquet", engine="pyarrow")
louisiana_sitios.to_parquet("../../Datasets/merge/louisiana_sitios.parquet", engine="pyarrow")